Using our [toolbox](http://neurolearn.readthedocs.io/en/latest/index.html) there's 2 main python classes that can help achieve ISC analysis:  

1) Brain_Data, which is a 1d or 2d representation of a nifti file. In the cases of something like an image with 1 value at each voxel (e.g beta map, T1, etc) this will be a single 1d vector of voxel values. In the case of something with more than 1 value at each voxel (e.g. fMRI time-series, concatenated beta-maps from multiple subjects), this will be a 2d vector of observations X  voxels.  

    - This class has a bunch of methods on it to do stuff like simple arithmetic operations, to more sophisticated things like regression, extracting a time-series from a mask, etc

2) Adjacency, which is a 1d or 2d representation of a distance matrix or several distance matrices. If theres just 1 distance matrix this will be a single 1d vector (i.e. flattened upper triangle if the matrix is symmetrical, or all values stretched out if it isn't). If there is more than 1 distance matrix this will be a 2d vector of observations X flattened matrix values.  

    - This class also has a bunch of methods just like Brain_Data

You'll need 2 things to do an ISC analysis:  

1) Nifti file(s) containing ROI masks within which to compute ISC - this can either be a single 4d file that several ROI masks within it, a single 3d file with just 1 ROI mask, a single 3d file within multiple ROIs indicated by different non-overlapping integer value at each voxel  

2) 4d Nifti files for all subjects to correlate

In [1]:
import nltools
from nltools.data import Brain_Data, Adjacency
from nltools.mask import expand_mask
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

import os

% matplotlib inline

print(nltools.__version__)

/Users/mdclark/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0.3.5


In [2]:
folder_path = "/Volumes/AdditionalStorage/ISC_Analysis"


# Useful stuff for dealing with the ROI mask(s)



# Single ROI mask in 3d file; just load it up
mask = Brain_Data(data = folder_path + '/masked_csv/vDMN.nii.gz') # this now a 1d array of voxels containing 1's and 0s

# Multiple ROI mask in 3d file (indicated by integer values at each voxel)
#mask = Brain_Data('path/to/mask.nii.gz')
# Expand it into a 2d array of mask X voxel
#mask = expand_mask(mask)

# Multiple ROI mask in 4d file; just load it up
#mask = Brain_Data('path/to/mask.nii.gz') # this is a 2d array of mask X voxels

/Volumes/AdditionalStorage/ISC_Analysis


In [3]:
#Lets say you want to compute the ISC within an ROI (i.e. the pair-wise correlation of subjects average time-series across the entire ROI)
#Below assumes you have just a single mask, but you can easily adapt this by adding another loop over ROIs, e.g. for roi in mask: ...

video = 6
mask_region = "vDMN"

subjects = [ '145', '157', '168', '184', '200', '214',  '219', '220', '222', '223', '229', '245', '253', '257', '263',  '267', '270', '277']
num_subs = len(subjects)

subject = os.path.join(folder_path, 'data', 'sub-' + subjects[0], 'func', 'video_' + str(video) + '.nii.gz')
num_TRs = Brain_Data(subject).shape()[0]
    


subject_timeseres = np.zeros((num_subs,num_TRs)) # initialize an empty matrix thats subjects X timeseries to save the extracted time-series 

for i, subject in enumerate(subjects):
    
    #try:
    
    subject_path = os.path.join(folder_path,'data', 'sub-' + subject, 'func', 'video_' + str(video) + '.nii.gz')
    print(subject_path)
    
    # Load in their data
    sub_dat = Brain_Data(subject_path)
    
    # Extract mean time-series from ROI
    sub_roi = sub_dat.extract_roi(mask)

    data_frame = pd.DataFrame(sub_roi)
    output = os.path.join(folder_path, 'masked_csv', mask_region, subject + '_video' + str(video) + 'mask.csv')

    data_frame.to_csv(output, header=None)

    #except:
     #   pass

NameError: name 'num_TRs' is not defined

In [ ]:
# Lets say you want to compute the ISC within an ROI (i.e. the pair-wise correlation of subjects average time-series across the entire ROI)
# Below assumes you have just a single mask, but you can easily adapt this by adding another loop over ROIs, e.g. for roi in mask: ...

mask_region = "vDMN"

subjects = [ '145', '157', '168', '184', '200', '214',  '219', '220', '222', '223', '229', '245', '253', '257', '263',  '267', '270', '277']
subject = os.path.join(folder_path, 'data', 'sub-' + subjects[1], 'func', 'video_' + str(video) + '.nii.gz')
num_subs = len(subjects)

num_TRs = Brain_Data(subject).shape()[0]

subject_timeseres = np.zeros((num_subs,num_TRs)) # initialize an empty matrix thats subjects X timeseries to save the extracted time-series 

for i, subject in enumerate(subjects):
    csv_path = os.path.join(folder_path, 'masked_csv', mask_region,  subject + "_video" + str(video) + "mask.csv")

    try: 
        # Load in their data
        sub_roi = pd.read_csv(csv_path, header=None)
        subject_timeseres[i,:] = sub_roi[1] # just grab the numpy array values 
    except: pass

# Now compute pairwise correlations 
sub_corrs = 1 - pairwise_distances(subject_timeseres,metric='correlation') # I'm subtracting by 1 because pairwise_distance gives you correlation distances (i.e. values from 0-2), this just converts from -1 to 1

# Save it as an adjacency matrix
sub_ISC = Adjacency(data=sub_corrs)

In [ ]:


sns.set(style="white")

# Compute the correlation matrix
corr = sub_corrs

# Genesampling_rate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))


sns.heatmap(corr, vmax=1, center=0,
            square=True, mask=mask, linewidths=.5, cbar_kws={"shrink": .5})

ax.set_xticklabels(subjects); 
ax.set_yticklabels(subjects); 


In [ ]:
#timeseries


plotcorrs = pd.DataFrame(subject_timeseres).transpose().reset_index()

plotcorrs = (plotcorrs - plotcorrs.mean()) / (plotcorrs.max() - plotcorrs.min())


plotcorrs = plotcorrs.loc[:, (plotcorrs != 0).any(axis=0)]


x = plotcorrs['index']
y = plotcorrs.drop(['index'], axis=1)

plt.figure(figsize=(14,4))
plt.style.use('ggplot')

plt.title("Video: " + str(video))
plt.ylabel('Rating (range 1-9)')
plt.xlabel('Time (in seconds)')



plt.plot(x, y, '--', linewidth=3, alpha=.7);



In [ ]:
mask = Brain_Data(folder_path + '/masked_csv/vDMN.nii.gz') # this now a 1d array of voxels containing 1's and 0s

mask.plot()

In [ ]:
subject = '157'
subject_path = os.path.join(folder_path,'data', 'sub-' + subject, 'func', 'video_' + str(video) + '.nii.gz')
print(subject_path)
# Load in their data
sub_dat = Brain_Data(subject_path)
sub_dat = sub_dat.scale()

In [ ]:
Brain_Data.scale(sub_dat)

Few other pointers:  

If you want to do ISC for multiple ROIs you'll probably want to do something like load in a single subject, loop over a `Brain_Data` object containing multiple ROIs, use the `extract` method to grab mean timeseries from each, then save all those to a single csv file for that subject using pandas or numpy. Then after you've done that for each subject just load in their csv files and proceed with correlating them as you want. This probably the fastest way to do it because very few computers can load say 30 subjects full time-series of a few minutes into RAM at once (unless maybe the time-series are very short). So this way just extracts all the ROIs for each subject first then lets you do whatever analyses you want on them. 